# Homework Assignment 2: Data Prep
In this homework assignment, you will continue your exploration of the [SWAN-SF Dataset](https://doi.org/10.7910/DVN/EBCFKM), described in the paper found [here](https://doi.org/10.1038/s41597-020-0548-x).


This assignment will have you explore the cardinalities, number of missing values, detect outliers, handle missing values and outliers, and create data quality report for original and cleaned dataset. Additionally, you will be asked to provide documentation for your functions.

## Step 1: Downloading the Data

This assignment will only be using [Partition 1](https://dataverse.harvard.edu/api/access/datafile/:persistentId?persistentId=doi:10.7910/DVN/EBCFKM/BMXYCB). Recall that in Homework 1 we started to construct the analytics base table for our [SWAN-SF Dataset](https://doi.org/10.7910/DVN/EBCFKM). In that assignment, we read the data from the two subdirectories, __FL__ and __NF__, of the __partition1__ direcotry. These two subdirectories represented the two classes of our target feature in the solar flare prediction problem we will be attempting to solve this semester. We then processed these samples of multivariate time series to construct descriptive features for each sample, and then placed them into our analytics base table.

In this assignment, you will be utilizing a set of extracted descriptive features much like what you were asked to construct in Homework 1. However, this dataset contains many more extracted features than you were asked to compute for Homework 1 (>800), so we need to explore the data to find data quality issues and identify ways to address these issues. Below are links to the full extracted feature dataset for partition 1 and a toy dataset to use for testing you functions.

__Note:__ Since the full dataset, and multiple copies of partially processed intermediary results, tend to take up a bit of space, you can use the toy dataset to implement and test your code. You may need to edit the data to fully test each of the requirements, but that is left as an exercise for the student. The full partition dataset is only included for those who wish to work with it once they have their code implemented. 

- [Full Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/partition1ExtractedFeatures.csv)
- [Toy Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/toy_partition1ExtractedFeatures.csv)

Now that you have the extracted features csv files, you will load that data into a Pandas DataFrame using the [pandas.read_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) method.  


In [9]:
import os
import pandas as pd
from pandas import DataFrame 
import numpy as np

### Read the dataset

In [10]:
data_dir = 'data/MVTS'
data_file = 'partition1ExtractedFeatures.csv'

In [11]:
file_name = os.path.join(data_dir, data_file).replace("\\", "/")
abt = pd.read_csv(file_name, index_col=0)

### Q1 (20 points)

Write a function to extract the various pieces of a data quality report, for a specific attribute, and return a dataframe with this information.

 * 'Feature Name': Contains the time series statistical feature name
 
 * 'Cardinality': Contains the count of unique values for the feature
            
 * 'Non-null Count': Contains the number of non-null entries for the feature
            
 * 'Null Count': Contains the number of null or missing entries for the feature
            
 * 'Min': Contains the minimum value of the feature (Without considering the null or nan value)
 
 * '25th': Contains the first quartile (25%) of the feature values (Without considering the null/nan value)
 
 * 'Mean': Contains the mean of the feature values (Without considering the null/nan value)
 
 * '50th': Contains the median of the feature values (Without considering the null/nan value)
            
 * '75th': Contains the third quartile (75%) of the feature values (Without considering the null/nan value)
 
 * 'Max': Contains the maximum value of the feature (Without considering the null/nan value),
            
 * 'Std. Dev': Contains the standard deviation of the feature (Without considering the null/nan value)
 
In addition to the values above, you should identify the number of upper and lower outliers using the $val < Q1-1.5IQR$ and $val > Q3+1.5IQR$ outlier identification method. These added features should be called `Outlier Count Low` and `Outliers Count High` respectively.


 
 Some useful functions for this can be found at:
 
 * [Numpy.percentile](https://numpy.org/doc/stable/reference/generated/numpy.percentile.html)
 
 * [pandas.isna](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.isna.html)
 
 * [Numpy.mean](https://numpy.org/doc/stable/reference/generated/numpy.mean.html)
 
 * [Numpy.std](https://numpy.org/doc/stable/reference/generated/numpy.std.html)
 

In [12]:
def calc_summary_for(feature_name:str, data:DataFrame) -> DataFrame:
    """
    Extract various pieces of information for a data quality report, and return the information for a single feature as a dataframe.
    
    Parameters:
    -----------
    feature_name: This is a string which contains the name of the feature for the specific type. 
    data: It is the input data from which we are supposed to extract information to perform computations on. It is in the form of a dataframe.
    The return type should also be a dataframe, but will have only 1 row, as we are processing only 1 feature.
    
    Output:
    -------
    It would be a dataframe with 1 row, as only 1 feature has been processed. The values of the feature have been stored in a numpy array.
    Features were then extracted using tolist(), std(), sum(), isnull(), and percentile() functions.
    All the features in the summary_feature_names list are included in here.
    """
    summary_feature_names = ['Feature Name', 'Cardinality', 'Non-null Count', 'Null Count', 'Min', '25th', 'Mean', 
                             '50th', '75th', 'Max', 'Std. Dev','Outlier Count Low', 'Outlier Count High']
    # Your answer to Q1 goes here!
    values = np.array(data[feature_name].tolist())
    values = values[~np.isnan(values)]
    cardinality = len(set(data[feature_name].tolist()))
    nnull_count = data[feature_name].notnull().sum()
    null_count = data[feature_name].isnull().sum()
    min_val = min(values)
    q1 = np.percentile(values, 25)
    mean = np.mean(values)
    q2 = np.percentile(values, 50)
    q3 = np.percentile(values, 75)
    max_val = max(values)
    std = np.std(values)
    iqr = q3 - q2
    outlier_count_low = (data[feature_name].dropna() < (q1 - 1.5*iqr)).sum()
    outlier_count_high = (data[feature_name].dropna() > (q3 + 1.5*iqr)).sum()
    info = [[feature_name, cardinality, nnull_count, null_count, min_val, q1, mean, q2, q3, max_val, std, outlier_count_low, outlier_count_high]]
    summary = pd.DataFrame(info, columns = summary_feature_names)
    return summary
    # Make sure to return a DataFrame with the features specified.
    pass

In [13]:
summary_TOTUSJH_min = calc_summary_for('TOTUSJH_min', abt)
summary_TOTUSJH_min

,Feature Name,Cardinality,Non-null Count,Null Count,Min,25th,Mean,50th,75th,Max,Std. Dev,Outlier Count Low,Outlier Count High
0,TOTUSJH_min,30165,73492,0,0.0,29.756125,396.697968,110.011893,418.614528,5680.232811,694.293786,0,10182


### Q2 (20 points)
Using what you produced to answere Q1, you should now write a function to construct the data quality report for all of the numerical features of our dataset.  You should loop over all of the features in the analytics base table represented by the input feature dataset files from partition 1, with the exception of the first column (this is the index column if you read the file correctly), and the `id`, `lab`, `st`, and `et` columns.  

Your output from this function will be a DataFrame that has 1 row for each feature. 

In [14]:
def construct_quality_report(data:DataFrame) -> DataFrame:
    """
    Constructs data quality report for all of the numerical features in the dataset. Unlike the previous function, this has all the rows in the file.
    
    Parameters:
    -----------
    data: This is a dataframe with all the input values from a file. It has about 73000 rows.
    
    Output:
    -------
    First, the excluded columns are deleted from the dataframe. Then, labels are stored in a list and dataframes are appeneded to one another.
    The final dataframe is returned after all the appending is done.
    """
    # Your answer to Q2 goes here!
    excluded_columns = ['id', 'lab', 'st', 'et']
    
    summary_feature_names = ['Feature Name', 'Cardinality', 'Non-null Count', 'Null Count', 'Min', '25th', 'Mean', 
                             '50th', '75th', 'Max', 'Std. Dev','Outlier Count Low', 'Outlier Count High']
    composite_data = pd.DataFrame()
    data = data.drop(columns = excluded_columns)
    features = list(data)
    for ind_feature in features:
        feature_data = calc_summary_for(ind_feature, abt)
        composite_data = composite_data.append(feature_data, ignore_index=True)
    return composite_data
    # Make sure to return a DataFrame with the features specified.
    pass

In [15]:
summary_table = construct_quality_report(abt)
print(summary_table.shape)  # checking the dimensionality is often a useful test.

(816, 13)


### Q3 (20 points)
#### Drop features with improper cardinality:
Using the quality report summary table that is returned from the function you wrote for Q2, we are now going to investigate our data. For this, you should use the table returned for the [Full Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/partition1ExtractedFeatures.csv) and not the toy dataset I provided for testing.

Since we are using real valued features, a majority of them shall have a cardinality close to the sample count. So, for this question, you are to write a function that takes in the summary table and the input dataset DataFrame, and drops the feature that have a cardinality less than 10. This feature should be dropped from both the data quality report summary table and from the actual input dataset DataFrame.

A useful method for this operation is:

* [pandas.DataFrame.drop](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html) (Make sure to use the inplace option otherwise it returns a copy)

In [16]:
def drop_low_card_data(summary_table:DataFrame, data:DataFrame) -> None:
    """
    Removing features that have a cardinality to less than 10, as it should have cardinality close to sample count.
    
    Parameters:
    -----------
    summary_table: This is the updated table with the dropped features being dropped. Contains the summary of all values and operations on the data.
    data: This is another dataframe type, just like summary_table. It is the original input set. The dropped_features list is applied here too, so that those values can be excluded.
    
    Output:
    -------
    There is no return type.
    This modifies 2 dataframes, with certain values removed. The values would be the ones with cardinality less than 10, as they have too many similar values.
    """
    # Your answer to Q3 goes here!
    dropped_features = []
    indicies = summary_table[summary_table['Cardinality'] < 10].index
    for index in list(indicies.values):
        dropped_features.append(summary_table.iloc[index]['Feature Name'])
    summary_table = summary_table.drop(indicies, inplace=True)
    data = data.drop(columns = dropped_features, inplace=True)
    pass

In [17]:
drop_low_card_data(summary_table, abt)
print(summary_table.shape)
print(abt.shape)

(801, 13)
(73492, 805)


### Q4 (20 points)
#### Drop features with excessive NaN

Again, using the quality report summary table that is returned from the function you wrote for Q2, we are going to continue investigating our data. For this, you should still be using the table returned for the [Full Partition 1 feature dataset](http://dmlab.cs.gsu.edu/solar/data/partition1ExtractedFeatures.csv) and not the toy dataset I provided for testing.

Like the features that were dropped for Q3, some of the extracted features don't work on all of the variates of the input multi-variate time series samples very well.  So, some of these features return an excessive number of `NaN` values.  These are not verry useful features, so we want to get rid of them before we continue. To do this, you are to write a function that takes in the summary table and the input dataset DataFrame, and drops the features that have **more than 1%** of the entries as null/nan values. Again, these features should be dropped from both the data quality report summary table and from the actual input dataset DataFrame.

As in Q3, a useful method for this operation is:

* [pandas.DataFrame.drop](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop.html) (Make sure to use the inplace option otherwise it returns a copy)

In [18]:
def drop_excessive_nan_data(summary_table:DataFrame, data:DataFrame) -> None:
    """
    Drops features that have nan values which account for more than 1% of the data. Does not return anything.
    
    Parameters:
    -----------
    summary_table: This is the updated table with the dropped features being dropped. Contains the summary of all values and operations on the data.
    The features with cardinality less than 10 have also been removed.
    data: This is another dataframe type, just like summary_table. It is the original input set. The dropped_features list is applied here too, so that those values can be excluded.
    The features with cardinality less than 10 have also been removed.
    
    Output:
    -------
    Although there is no return type, this modifies the 2 dataframes and updates them to contain values that have features with cardinality greater than or equal to 10 and nan percentage less than 1.
    """
    # Your answer to Q4 goes here!
    features = []
    for column in list(data):
        if(((data[column].isna().sum())/len(set(data[column].tolist()))) > 0.01):
            features.append(column)
    indicies = []
    for feature in features:
        feature_index = summary_table.index[summary_table['Feature Name'] == feature]
        for each in list(feature_index.values):
            indicies.append(each)
    summary_table = summary_table.drop(indicies, inplace=True)
    data = data.drop(columns = features, inplace=True)
    pass

In [19]:
drop_excessive_nan_data(summary_table, abt)
print(summary_table.shape)
print(abt.shape)

(799, 13)
(73492, 803)


Now that you have the data cleaned up a little, save the results of both your summary table and your analytics base table using the [pandas.to_csv](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_csv.html) method. We will want to use these results for the next homework assignment.

In [20]:
out_dir = 'data/MVTS'
out_summary_table_name = 'data_quality_table.csv'
out_abt_name = 'cleaned_partition1ExtractedFeatures.csv'

In [21]:
# Save here!
summary_table.to_csv(os.path.join(out_dir, out_summary_table_name).replace("\\", "/"))
abt.to_csv(os.path.join(out_dir, out_abt_name).replace("\\", "/"))

### Q5 (20 points)
#### Add docstring to your functions

Let's revisit our programming skill while learning Fundamentals of Data Science. 

Your code is only as valuable as its reusability. Without understandable and legible documentation (which makes maintainability and reusability possible) nobody would like to use your code, let alone to pay for it. ;)

If you want to know more about the value of documentation, read [this article](https://www.freecodecamp.org/news/why-documentation-matters-and-why-you-should-include-it-in-your-code-41ef62dd5c2f/). There are even conferences on this topic; see [this website](https://www.writethedocs.org/guide/writing/beginners-guide-to-docs/).

In Python, the documentation that is embedded in the code is called **docstring**. In the example below, the "string" wrapped in triple quotes is there to tell us all about this function.

In [22]:
def nanmean(a, axis=None):
    """
    Compute the arithmetic mean along the specified axis, ignoring NaNs.
    
    Parameters
    ----------
    a : array_like
        Array containing numbers whose mean is desired. If `a` is not an
        array, a conversion is attempted.
    axis : {int, tuple of int, None}, optional
        Axis or axes along which the means are computed. The default is to compute
        the mean of the flattened array.
    """
    # some magic happens here that we don't care about.
    pass

Note that this is not just a *comment*. If you execute `nanmean` and then call it (as if you want to use it), you can hit `shift+Tab` while your cursor is on the function name, and see how the docstring gets compiled and then pops up. This allows other users to see our description even when they don't have access to our source code. Try it! You can do this with other NumPy and Pandas functions/methods that you've been using.

The above example is a simplified version of the method `nanmean` copied from the NumPy library ([here](https://github.com/numpy/numpy/blob/v1.21.0/numpy/lib/nanfunctions.py#L862-L957)). Feel free to check out their complete docstrings.


Your last task is to provide docstrings for the 4 methods you've implemented. Simply go back to those cells and modify your functions. Feel free to use the text provided to you (in the assignment descriptions) to enrich your docstrings. Keep in mind that your docstring needs (1) a general description, (2) a short description for each input, and (3) a short description for the output.

How to check your docstring? Hit `shift+Tab` and see if the pop-up message is correctly compiled, and make sure your description answers all the questions about your functions.